# __Libraries__

In [251]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import plotly.express as px

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# __Loading data__

In [252]:
# https://www.investing.com/currencies/usd-cop-historical-data
df = pd.read_csv("/content/USD_COP1.csv")

# __Clearing data__

In [253]:
df = df.sort_values(by='Date')
df = df.set_index('Date')
df.head()

,Price,Open,High,Low,Change
Date,,,,,
2004-01-02,2776.65,2780.55,2781.05,2776.55,-0.0012
2004-01-05,2770.25,2775.65,2783.45,2768.25,-0.0023
2004-01-06,2758.00,2771.20,2772.60,2757.70,-0.0044
2004-01-07,2763.00,2751.00,2764.60,2750.20,0.0018
2004-01-08,2751.50,2763.20,2764.20,2740.90,-0.0042


# __Data exploration__

In [254]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4905 entries, 2004-01-02 to 2022-10-21
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Price   4905 non-null   float64
 1   Open    4905 non-null   float64
 2   High    4905 non-null   float64
 3   Low     4905 non-null   float64
 4   Change  4905 non-null   float64
dtypes: float64(5)
memory usage: 229.9+ KB


In [255]:
df.describe()

,Price,Open,High,Low,Change
count,4905.000000,4905.000000,4905.000000,4905.000000,4905.000000
mean,2588.442118,2588.119307,2601.888255,2574.575609,0.000147
std,705.260860,704.663794,710.653533,699.443379,0.007855
min,1656.500000,1656.750000,1670.700000,1635.300000,-0.061500
25%,1935.200000,1935.490000,1943.500000,1927.650000,-0.003600
50%,2378.200000,2378.300000,2390.500000,2366.750000,0.000000
75%,3056.570000,3055.810000,3074.750000,3036.370000,0.003700
max,4906.350000,4904.000000,4960.130000,4877.550000,0.062300


In [256]:
fig = px.line(df[['Price', 'Open', 'High', 'Low']], 
              title = "U.S. DOLLAR / COLOMBIAN PESO",
              labels={
                     "Date": "Time",
                     "value": "Value",
                     "variable": ""
                 },
              template="plotly_white",
              color_discrete_sequence=px.colors.qualitative.G10,
              width=900, height=500,
              )
fig.update_layout(margin=dict(l=100, b=80, r=120, t=100))
fig.show()

In [257]:
fig = px.line(df[['Change']], 
              title = "U.S. DOLLAR / COLOMBIAN PESO",
              labels={
                     "Date": "Time",
                     "value": "Value",
                     "variable": ""
                 },
              template="plotly_white",
              color_discrete_sequence=px.colors.qualitative.G10,
              width=900, height=500,
              )
fig.update_layout(margin=dict(l=100, b=80, r=120, t=100))
fig.show()

In [338]:
df_price = df['Price']

In [339]:
fig = px.line(df_price, 
              title = "U.S. DOLLAR / COLOMBIAN PESO",
              labels={
                     "Date": "Time",
                     "value": "Value",
                     "variable": ""
                 },
              template="plotly_white",
              color_discrete_sequence=px.colors.qualitative.G10,
              width=900, height=500,
              )
fig.update_layout(margin=dict(l=100, b=80, r=120, t=100))
fig.show()

# __Dataset__

## Split DataFrame


In [340]:
training_size = int(len(df_price)*0.80)
test_size = len(df_price)-training_size
df_train_price = df_price.reset_index().iloc[:training_size,:]
df_test_price = df_price.reset_index().iloc[training_size:len(df_price),]

## Split np.array

In [341]:
scaler=MinMaxScaler(feature_range=(0,1))
df_price=scaler.fit_transform(np.array(df_price).reshape(-1,1))
print(df_price.shape)

(4905, 1)


In [342]:
training_size=int(len(df_price)*0.80)
test_size=len(df_price)-training_size
train_data,test_data=df_price[0:training_size,:],df_price[training_size:len(df_price),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (3924, 1)
test_data:  (981, 1)


## Create DataSet

In [343]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [344]:
time_step = 6
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (3917, 6)
y_train:  (3917,)
X_test:  (974, 6)
y_test (974,)


In [345]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (3917, 6, 1)
X_test:  (974, 6, 1)


# __Model and Training__

In [346]:
model=Sequential()
model.add(LSTM(1000,input_shape=(None,1),activation="relu"))
model.add(Dense(500))
model.add(Dense(1))
model.compile(loss="mean_squared_error",optimizer="adam")

In [347]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
123/123 [==============================] - 30s 237ms/step - loss: 0.0013 - val_loss: 2.5748e-04
Epoch 2/200
123/123 [==============================] - 29s 234ms/step - loss: 7.9670e-05 - val_loss: 2.8267e-04
Epoch 3/200
123/123 [==============================] - 29s 234ms/step - loss: 6.6915e-05 - val_loss: 5.5783e-04
Epoch 4/200
123/123 [==============================] - 29s 235ms/step - loss: 7.8103e-05 - val_loss: 1.8566e-04
Epoch 5/200
123/123 [==============================] - 29s 235ms/step - loss: 6.4203e-05 - val_loss: 1.9380e-04
Epoch 6/200
123/123 [==============================] - 29s 236ms/step - loss: 6.9733e-05 - val_loss: 1.6873e-04
Epoch 7/200
123/123 [==============================] - 29s 236ms/step - loss: 7.9298e-05 - val_loss: 2.0362e-04
Epoch 8/200
123/123 [==============================] - 29s 236ms/step - loss: 4.6643e-05 - val_loss: 1.2419e-04
Epoch 9/200
123/123 [==============================] - 29s 236ms/step - loss: 4.7147e-05 - val_loss: 5.8252e

# __Evaluation__

In [348]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

In [349]:
fig = px.line(pd.DataFrame({'Loss':loss, 'Val Loss':val_loss}, index=epochs),
              title = "Training and validation loss",
              labels={
                     "value": "",
                     "index": "Epoch",
                     "variable": ""
                 },
              template="plotly_white",
              color_discrete_sequence=px.colors.qualitative.G10,
              width=900, height=500,
              )
fig.update_layout(margin=dict(l=100, b=80, r=120, t=100))
fig.show()

In [350]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
train_predict.shape, test_predict.shape

31/31 [==============================] - 2s 53ms/step


((3917, 1), (974, 1))

In [351]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [352]:
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  20.1350745908849
Train data MSE:  405.4212287804988
Train data MAE:  13.829699660284895
-------------------------------------------------------------------------------------
Test data RMSE:  38.25325507163687
Test data MSE:  1463.311523575712
Test data MAE:  26.654917102476908


In [353]:
print("Train data explained variance regression score:", 
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", 
      explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9981041787639715
Test data explained variance regression score: 0.9881300221327449


In [354]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9980480989947772
Test data R2 score: 0.9876046320660044


In [355]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  6.527778887309377e-05
Test data MGD:  9.505519616732326e-05
----------------------------------------------------------------------
Train data MPD:  0.15968993370474785
Test data MPD:  0.37108627079577367


In [356]:
test_predict.shape

(974, 1)

In [315]:
df_test_price_new = df_test_price.iloc[5:1961,].set_index('Date').copy()
df_test_price_new['Forecast'] = test_predict

In [357]:
fig = px.line(df_test_price_new, 
              title = "Forecast: U.S. DOLLAR / COLOMBIAN PESO",
              labels={
                     "Date": "Time",
                     "value": "Value",
                     "variable": ""
                 },
              template="plotly_white",
              color_discrete_sequence=px.colors.qualitative.G10,
              width=900, height=500,
              )
fig.update_layout(margin=dict(l=100, b=80, r=120, t=100))
fig.show()